[条件随机场介绍（译）Introduction to Conditional Random Fields](https://zhuanlan.zhihu.com/p/25558273)

In [21]:
###### 0.数据 ########
import numpy as np
import jieba
import jieba.posseg as pseg

data = '猎犬在池塘游泳'
data_split =jieba.cut(data,cut_all=False,HMM=False)
data_str = " /".join(data_split)
print(data_str)
print('------------')
#看看jieba自带的词性标注例子
result = pseg.cut(data)  
for w in  result:
    print(w.word, "/", w.flag, ", ",)

data_list = data_str.split(" /")   
print(data_list)    

猎犬 /在 /池塘 /游泳
------------
猎犬 / n , 
在 / p , 
池塘 / ns , 
游泳 / vn , 
['猎犬', '在', '池塘', '游泳']


In [29]:
###### 1.模型 ########
'''
为了简化，假设只有n，v，p三种词性
'''
#特征函数
def ff1(s,i,li,li_pre):
    if  i == 0 and li == "n":
        return 1
    else:return 0
    
def ff2(s,i,li,li_pre):
    if i!=0 and ((li == 'n' and li_pre !='n') or (li != 'n' and li_pre =='n')):
        return 1
    else:return 0
    
def ff3(s,i,li,li_pre):
    if i!=0 and ((li == 'v' and li_pre !='v') or (li != 'v' and li_pre =='v')):
        return 1
    else:return 0
    
def ff4(s,i,li,li_pre):
    if i!=0 and ((li == 'p' and li_pre !='p') or (li != 'p' and li_pre =='p')):
        return 1
    else:return 0    

def ff5(s,i,li,li_pre):
    if i==-1 and (li != 'p' or (li != 'v' and li_pre =='p')):
        return 1
    else:return 0  

def ff6(s,i,li,li_pre):
    if i==-1 and ((li == 'n' and li_pre =='v') or (li == 'v' and li_pre =='n')):
        return 1
    else:return 0  
    

featureFunc = [ff1,ff2,ff3,ff4,ff5,ff6]
m = len(featureFunc)
#权重 随机初始化
lambd = np.random.rand(m)
print(lambd)
#训练数据
s = [['青蛙','在','夜晚','唱歌'],
     ['苹果','是','水果'],
     ['他','喜欢','足球'],]
#防止li_pre取到-1，加一个0
l = [[0,'n','p','n','v'],
     [0,'n','v','n'],
     [0,'p','v','n'],]

[ 0.17281421  0.54401201  0.65880065  0.2888991   0.17612854  0.81668556]


In [39]:
###### 2.权重学习 ########
alpha = 0.1
iterations = 10
it = 0

while it <iterations:
    g1 = 0
    for i in range(len(s)):#对每个训练句子
        for k in range(len(s[i])):#对每个单词
            for j in range(m): #对每个函数
                f = featureFunc[j]
                #lmbd = lambd[j]
                g1+=f(s[i], k, l[i][k+1], l[i][k])
    
    g2 = 0
    p = 0
    for i in range(len(s)):#对每个训练句子
        for k in range(len(s[i])):#对每个单词
            for j in range(m): #对每个函数
                f = featureFunc[j]
                lmbd = lambd[j]
                g2+=f(s[i],k,l[i][k+1],l[i][k])*lmbd
######这里还是有问题，原公式还有一个P（l'|s）  求和，不知道什么意思 #######################         
                #p = 
    #g2=p*g2        
    grad =g1-g2 
    lambd += alpha*grad
    it+=1
print(lambd)

[ 0.68716415  1.05836194  1.17315059  0.80324904  0.69047848  1.3310355 ]


In [58]:
###### 3.找到最佳标注 ########
#直接为每个可能的标注复杂度为指数级，更好的办法是使（线性链式）CRF满足最佳子结构（optimal substructure）的属性，可以采用DP
#这里为了简化直接用暴力方法
poss = []
typeW = ['n','v','p']
for i in typeW:
    for j in typeW:
        for k in typeW:
            for n in typeW:
                poss.append([i,j,k,n])
                
maxS = 0
maxI = 0
for i in range(len(poss)): 
    value = 0
    for j in range(m): #外层循环，遍历所有特征函数
        s = [0]+poss[i]
        for w in range(len(s)-1):
            value+=featureFunc[j](poss[i],w,s[w+1],s[w])*lambd[j]
    if value>maxS:
        maxS = value
        maxI = i
        
print('-----')
print(maxS)
print(poss[maxI])
                           

-----
7.38170173798
['n', 'v', 'n', 'v']


正确应该是[n,p,n,v],这里数据集比较小，特征函数也很少，所以结果勉强可以。